## Submission

In [1]:
# %load mask_to_submission.py
#!/usr/bin/env python3

import os
import numpy as np
import matplotlib.image as mpimg
import re

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


if __name__ == '__main__':
    submission_filename = 'submission.csv'
    image_filenames = []
    for i in range(1, 51):
        image_filename = 'training/groundtruth/satImage_' + '%.3d' % i + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)


training/groundtruth/satImage_001.png
training/groundtruth/satImage_002.png
training/groundtruth/satImage_003.png
training/groundtruth/satImage_004.png
training/groundtruth/satImage_005.png
training/groundtruth/satImage_006.png
training/groundtruth/satImage_007.png
training/groundtruth/satImage_008.png
training/groundtruth/satImage_009.png
training/groundtruth/satImage_010.png
training/groundtruth/satImage_011.png
training/groundtruth/satImage_012.png
training/groundtruth/satImage_013.png
training/groundtruth/satImage_014.png
training/groundtruth/satImage_015.png
training/groundtruth/satImage_016.png
training/groundtruth/satImage_017.png
training/groundtruth/satImage_018.png
training/groundtruth/satImage_019.png
training/groundtruth/satImage_020.png
training/groundtruth/satImage_021.png
training/groundtruth/satImage_022.png
training/groundtruth/satImage_023.png
training/groundtruth/satImage_024.png
training/groundtruth/satImage_025.png
training/groundtruth/satImage_026.png
training/gro

In [2]:
# %load submission_to_mask.py
#!/usr/bin/python
import os
import sys
from PIL import Image
import math
import matplotlib.image as mpimg
import numpy as np

label_file = 'submission.csv'

h = 16
w = h
imgwidth = int(math.ceil((600.0/w))*w)
imgheight = int(math.ceil((600.0/h))*h)
nc = 3

# Convert an array of binary labels to a uint8
def binary_to_uint8(img):
    rimg = (img * 255).round().astype(np.uint8)
    return rimg

def reconstruct_from_labels(image_id):
    im = np.zeros((imgwidth, imgheight), dtype=np.uint8)
    f = open(label_file)
    lines = f.readlines()
    image_id_str = '%.3d_' % image_id
    for i in range(1, len(lines)):
        line = lines[i]
        if not image_id_str in line:
            continue

        tokens = line.split(',')
        id = tokens[0]
        prediction = int(tokens[1])
        tokens = id.split('_')
        i = int(tokens[1])
        j = int(tokens[2])

        je = min(j+w, imgwidth)
        ie = min(i+h, imgheight)
        if prediction == 0:
            adata = np.zeros((w,h))
        else:
            adata = np.ones((w,h))

        im[j:je, i:ie] = binary_to_uint8(adata)

    Image.fromarray(im).save('prediction_images/prediction_' + '%.3d' % image_id + '.png')

    return im

for i in range(1, 5):
    reconstruct_from_labels(i)
   
